In [1]:
import os
from transformers import LlamaForCausalLM, LlamaTokenizer, BitsAndBytesConfig
from itertools import chain
from transformers import default_data_collator, Trainer, TrainingArguments, TrainerCallback
from trl import SFTTrainer
import torch
import datasets
from peft import (
    get_peft_model,
    LoraConfig,
    TaskType,
    prepare_model_for_int8_training,
    AutoPeftModelForCausalLM
)
import datetime

/home/edgar-pino/.pyenv/versions/3.10.4/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/edgar-pino/.pyenv/versions/3.10.4/envs/llm/lib/python3.10/site-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [5]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

In [6]:
torch.cuda.empty_cache()

In [7]:
version_date = int(datetime.datetime.now().timestamp())
model_id="./models/7B-hf"
device_map = "auto"
output_dir = f"./llama-2/{version_date}"
final_checkpoint_dir = os.path.join(output_dir, "final_checkpoint")


In [8]:
tokenizer = LlamaTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token


In [6]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = LlamaForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, use_cache=False, device_map=device_map)
model.config.use_cache = False
model.config.pretraining_tp = 1
model = prepare_model_for_int8_training(model)


Loading checkpoint shards: 100%|██████████| 2/2 [00:38<00:00, 19.21s/it]
Some weights of LlamaForCausalLM were not initialized from the model checkpoint at ./models/7B-hf and are newly initialized: ['model.layers.23.self_attn.rotary_emb.inv_freq', 'model.layers.20.self_attn.rotary_emb.inv_freq', 'model.layers.28.self_attn.rotary_emb.inv_freq', 'model.layers.30.self_attn.rotary_emb.inv_freq', 'model.layers.18.self_attn.rotary_emb.inv_freq', 'model.layers.16.self_attn.rotary_emb.inv_freq', 'model.layers.31.self_attn.rotary_emb.inv_freq', 'model.layers.10.self_attn.rotary_emb.inv_freq', 'model.layers.6.self_attn.rotary_emb.inv_freq', 'model.layers.14.self_attn.rotary_emb.inv_freq', 'model.layers.3.self_attn.rotary_emb.inv_freq', 'model.layers.11.self_attn.rotary_emb.inv_freq', 'model.layers.29.self_attn.rotary_emb.inv_freq', 'model.layers.15.self_attn.rotary_emb.inv_freq', 'model.layers.26.self_attn.rotary_emb.inv_freq', 'model.layers.17.self_attn.rotary_emb.inv_freq', 'model.layers.7.sel

In [7]:
eval_prompt = """
Implement the following program given the instructions:
Write a simple program to calculate the cosine similarity between two vectors in Typescript.

Code:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

/home/edgar-pino/.pyenv/versions/3.10.4/envs/llm/lib/python3.10/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
/home/edgar-pino/.pyenv/versions/3.10.4/envs/llm/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")



Implement the following program given the instructions:
Write a simple program to calculate the cosine similarity between two vectors in Typescript.

Code:
```typescript
function cosineSimilarity(vector1: Vector, vector2: Vector): number {
  const dotProduct = vector1.dot(vector2);
  const magnitude1 = vector1.magnitude();
  const magnitude2 = vector2.magnitude();
  return dotProduct / (magnitude1 * magnitude2);
}
```

Explanation:

The cosine similarity between two vectors is the cos


In [7]:
def apply_prompt_template(samples, tokenizer):
    prompt = (
        f"Implement the following program given the instructions:\n{{instructions}}\n---\nCode:\n{{code}}{{eos_token}}"
    )    
    
    batch_text = []

    batch = list(zip(samples['prompt'], samples['response']))

    for sample in batch:
        text = prompt.format(
            instructions=sample[0],
            code=sample[1],
            eos_token=tokenizer.eos_token,
        )
        batch_text.append(text)
    
    return {"text": batch_text}

In [8]:
def get_preprocessed_codes(tokenizer, split):
    dataset = datasets.load_dataset("nampdn-ai/tiny-codes", split=split)
    dataset = dataset.shuffle().select(range(1_000))

    def apply_prompt_template_batch(samples):
        return apply_prompt_template(samples, tokenizer)

    return dataset.map(apply_prompt_template_batch, batched=True, remove_columns=list(dataset.features))


In [9]:
dataset = get_preprocessed_codes(tokenizer, "train")

Map: 100%|██████████| 150000/150000 [00:06<00:00, 23761.52 examples/s]


In [10]:
output_dir = "./llama-coder"
final_checkpoint_dir = os.path.join(output_dir, "final_checkpoint")

In [11]:
max_seq_length = 1024

# Define training args
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    fp16=True,  # Use fp16 if available
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=50,
    save_strategy="no",
    optim="adamw_torch_fused",
    max_steps=10,
    report_to="none",
    learning_rate=1e-4,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    per_device_eval_batch_size=2,
    gradient_checkpointing= False
)

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules = ["q_proj", "v_proj"]
)

# https://huggingface.co/docs/trl/main/en/sft_trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_args,
    callbacks=[]
)

# Start training
trainer.train()

/home/edgar-pino/.pyenv/versions/3.10.4/envs/llm/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map: 100%|██████████| 150000/150000 [08:46<00:00, 284.68 examples/s]


Step,Training Loss
50,1.128200
100,0.785100
150,0.725400
200,0.694700
250,0.702900
300,0.663300
350,0.678200
400,0.661100
450,0.658400
500,0.654700


TrainOutput(global_step=500, training_loss=0.7352132568359375, metrics={'train_runtime': 5647.0774, 'train_samples_per_second': 0.354, 'train_steps_per_second': 0.089, 'total_flos': 2.7725034962386944e+16, 'train_loss': 0.7352132568359375, 'epoch': 0.01})

In [12]:
trainer.model.save_pretrained(final_checkpoint_dir)